# My name: Bryan Lihardinata
## Assignment Name: CA06 - kNN Recommendation System

# Program Inititialization Section
## Enter your import packages here

In [138]:
# import packages 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

The data contains thirty movies, including data for each movie across seven genres and their IMDB
ratings. The labels column values are all zeroes because we aren’t using this data set for classification or
regression. You can ignore this column. The implementation assumes that all columns contain numerical
data.

Additionally, there are relationships among the movies that will not be accounted for (e.g. actors,
directors, and themes) when using the KNN algorithm simply because the data that captures those
relationships are missing from the data set.
###### 0 = No, 1 = Yes

In [139]:
#read datasets
df = pd.read_csv('https://github.com/ArinB/CA05-kNN/raw/master/movies_recommendation_data.csv')
df.head()

,Movie ID,Movie Name,IMDB Rating,Biography,Drama,Thriller,Comedy,Crime,Mystery,History,Label
0,58,The Imitation Game,8.0,1,1,1,0,0,0,0,0
1,8,Ex Machina,7.7,0,1,0,0,0,1,0,0
2,46,A Beautiful Mind,8.2,1,1,0,0,0,0,0,0
3,62,Good Will Hunting,8.3,0,1,0,0,0,0,0,0
4,97,Forrest Gump,8.8,0,1,0,0,0,0,0,0


# Initial Data Investigation Section

## Summarized details
### Generate descriptive statistics that summarize the central tendency, dispersion, and shape of a dataset’s distribution, excluding NaN values.




In [140]:
#Statistical Description of data (data.describe)
df.describe()

,Movie ID,IMDB Rating,Biography,Drama,Thriller,Comedy,Crime,Mystery,History,Label
count,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.0
mean,48.133333,7.696667,0.233333,0.600000,0.100000,0.100000,0.133333,0.100000,0.100000,0.0
std,29.288969,0.666169,0.430183,0.498273,0.305129,0.305129,0.345746,0.305129,0.305129,0.0
min,1.000000,5.900000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
25%,27.750000,7.300000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
50%,48.500000,7.750000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
75%,64.250000,8.175000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
max,98.000000,8.800000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0


In [141]:
#Statistical Description of data (data.shape)
df.shape

(30, 11)

In [142]:
#Display number of non-null values for each column (data.count)
df.count()
#no null value

Movie ID       30
Movie Name     30
IMDB Rating    30
Biography      30
Drama          30
Thriller       30
Comedy         30
Crime          30
Mystery        30
History        30
Label          30
dtype: int64

In [143]:
#double Checking null value
df[df.isnull().any(axis=1)].count()

Movie ID       0
Movie Name     0
IMDB Rating    0
Biography      0
Drama          0
Thriller       0
Comedy         0
Crime          0
Mystery        0
History        0
Label          0
dtype: int64

In [144]:
#Display range, column, number of non-null objects of each column, datatype and memory usage (data.info)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Movie ID     30 non-null     int64  
 1   Movie Name   30 non-null     object 
 2   IMDB Rating  30 non-null     float64
 3   Biography    30 non-null     int64  
 4   Drama        30 non-null     int64  
 5   Thriller     30 non-null     int64  
 6   Comedy       30 non-null     int64  
 7   Crime        30 non-null     int64  
 8   Mystery      30 non-null     int64  
 9   History      30 non-null     int64  
 10  Label        30 non-null     int64  
dtypes: float64(1), int64(9), object(1)
memory usage: 2.7+ KB


In [145]:
#Display Top 10 and Bottom 10 records (head and tail)
df.head(10)

,Movie ID,Movie Name,IMDB Rating,Biography,Drama,Thriller,Comedy,Crime,Mystery,History,Label
0,58,The Imitation Game,8.0,1,1,1,0,0,0,0,0
1,8,Ex Machina,7.7,0,1,0,0,0,1,0,0
2,46,A Beautiful Mind,8.2,1,1,0,0,0,0,0,0
3,62,Good Will Hunting,8.3,0,1,0,0,0,0,0,0
4,97,Forrest Gump,8.8,0,1,0,0,0,0,0,0
5,98,21,6.8,0,1,0,0,1,0,1,0
6,31,Gifted,7.6,0,1,0,0,0,0,0,0
7,3,Travelling Salesman,5.9,0,1,0,0,0,1,0,0
8,51,Avatar,7.9,0,0,0,0,0,0,0,0
9,47,The Karate Kid,7.2,0,1,0,0,0,0,0,0


In [146]:
df.tail(10)

,Movie ID,Movie Name,IMDB Rating,Biography,Drama,Thriller,Comedy,Crime,Mystery,History,Label
20,57,The DaVinci Code,6.6,0,0,1,0,0,1,0,0
21,57,Stand and Deliver,7.3,0,1,0,0,0,0,0,0
22,14,The Terminator,8.0,0,0,0,0,0,0,0,0
23,69,21 Jump Street,7.2,0,0,0,1,1,0,0,0
24,98,The Avengers,8.1,0,0,0,0,0,0,0,0
25,17,Thor: Ragnarok,7.9,0,0,0,1,0,0,0,0
26,12,Spirit: Stallion of the Cimarron,7.1,0,0,0,0,0,0,0,0
27,1,Hacksaw Ridge,8.2,1,1,0,0,0,0,1,0
28,86,12 Years a Slave,8.1,1,1,0,0,0,0,1,0
29,46,Queen of Katwe,7.4,1,1,0,0,0,0,0,0


In [147]:
#The labels column values are all zeroes because we aren’t using this data set for classification or regression.
df = df.drop('Label', axis= 'columns')
df.head()

,Movie ID,Movie Name,IMDB Rating,Biography,Drama,Thriller,Comedy,Crime,Mystery,History
0,58,The Imitation Game,8.0,1,1,1,0,0,0,0
1,8,Ex Machina,7.7,0,1,0,0,0,1,0
2,46,A Beautiful Mind,8.2,1,1,0,0,0,0,0
3,62,Good Will Hunting,8.3,0,1,0,0,0,0,0
4,97,Forrest Gump,8.8,0,1,0,0,0,0,0


In [148]:
#Adding "The Post" into the dataset
#IMDB Rating = 7.2, Biography = Yes, Drama = Yes, Thriller = No, Comedy = No, Crime = No, Mystery = No, History = Yes
df.loc[len(df.index)] = [100, 'The Post', 7.2, 1, 1, 0, 0, 0, 0, 1] 
df.tail()

,Movie ID,Movie Name,IMDB Rating,Biography,Drama,Thriller,Comedy,Crime,Mystery,History
26,12,Spirit: Stallion of the Cimarron,7.1,0,0,0,0,0,0,0
27,1,Hacksaw Ridge,8.2,1,1,0,0,0,0,1
28,86,12 Years a Slave,8.1,1,1,0,0,0,0,1
29,46,Queen of Katwe,7.4,1,1,0,0,0,0,0
30,100,The Post,7.2,1,1,0,0,0,0,1


# Designed dataframes for recommendation system

## Splitting the dataframe into movie index + title and features

In [149]:
movie_idx = df.loc[:,['Movie ID','Movie Name']]
movie_idx.head()

,Movie ID,Movie Name
0,58,The Imitation Game
1,8,Ex Machina
2,46,A Beautiful Mind
3,62,Good Will Hunting
4,97,Forrest Gump


##Pivot Movie Feature

In [150]:
#Pivot Rating into Movie Feature
movie_feature = df.iloc[:, [0,2,3,4,5,6,7,8,9]]
movie_feature = movie_feature.set_index('Movie ID')
movie_feature.head()

,IMDB Rating,Biography,Drama,Thriller,Comedy,Crime,Mystery,History
Movie ID,,,,,,,,
58,8.0,1,1,1,0,0,0,0
8,7.7,0,1,0,0,0,1,0
46,8.2,1,1,0,0,0,0,0
62,8.3,0,1,0,0,0,0,0
97,8.8,0,1,0,0,0,0,0


In [151]:
#sparse matrices contain mostly zero values and are distinct from dense matrices
from scipy.sparse import csr_matrix
match_features = csr_matrix(movie_feature)
print(match_features)

#How to read: (row#, column# with non-zero value)    non-zero value

  (0, 0)	8.0
  (0, 1)	1.0
  (0, 2)	1.0
  (0, 3)	1.0
  (1, 0)	7.7
  (1, 2)	1.0
  (1, 6)	1.0
  (2, 0)	8.2
  (2, 1)	1.0
  (2, 2)	1.0
  (3, 0)	8.3
  (3, 2)	1.0
  (4, 0)	8.8
  (4, 2)	1.0
  (5, 0)	6.8
  (5, 2)	1.0
  (5, 5)	1.0
  (5, 7)	1.0
  (6, 0)	7.6
  (6, 2)	1.0
  (7, 0)	5.9
  (7, 2)	1.0
  (7, 6)	1.0
  (8, 0)	7.9
  (9, 0)	7.2
  :	:
  (21, 0)	7.3
  (21, 2)	1.0
  (22, 0)	8.0
  (23, 0)	7.2
  (23, 4)	1.0
  (23, 5)	1.0
  (24, 0)	8.1
  (25, 0)	7.9
  (25, 4)	1.0
  (26, 0)	7.1
  (27, 0)	8.2
  (27, 1)	1.0
  (27, 2)	1.0
  (27, 7)	1.0
  (28, 0)	8.1
  (28, 1)	1.0
  (28, 2)	1.0
  (28, 7)	1.0
  (29, 0)	7.4
  (29, 1)	1.0
  (29, 2)	1.0
  (30, 0)	7.2
  (30, 1)	1.0
  (30, 2)	1.0
  (30, 7)	1.0


# Part 1: Build a recommendation system

In [152]:
from sklearn.neighbors import NearestNeighbors

sklearn.neighbors.NearestNeighbors(*, n_neighbors=5, radius=1.0, algorithm='auto', leaf_size=30, metric='minkowski', p=2, metric_params=None, n_jobs=None

- n_neighbors = number of the nearest neighbors
- radius = Range of parameter space
- algorithm:
 - brute = Computation of ALL Euclidian Distance Pairs and vote/mean by
the k nearest neighbors
   - Based on eucladian (think pythagorean theorem)
 - kd_tree = A “tree based” approach to reduce the computational
inefficiencies of Brute Force approach
   - Example: If point A is very distant from point B, and point B is very close to point C, then
we know that points A and C are very distant, without having to explicitly
calculate their distance
 - ball_tree = to overcome the inefficiencies of K-D Tree for higher dimensions
   - A single distance calculation between the test point and the sphere centroid can
determine the range of distance w.r.t ALL points inside the sphere
 - auto = decide the most appropriate algorithm based on the values passed to fit method.
- leaf_size = belongs to kd_tree and ball_tree -> specifies the number of points at which the tree switches to brute-force, and encodes the tradeoff between the cost of accessing a node, and the cost of computing the distance function
- p = Parameter for the Minkowski metric
  - p=1 -> manhattan distance
  - p=2 -> eucladian distance


In [153]:
#since we want to know 5 movies similar to X, m_neighbors = 5
model_knn = NearestNeighbors(algorithm='brute', n_neighbors=5)
model_knn

NearestNeighbors(algorithm='brute', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [154]:
#train our features
model_knn.fit(match_features)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

# Part 2: Find 5 movies that are most similar to The Post
Following is the genre information about the movie “The Post”

IMDB Rating = 7.2, Biography = Yes, Drama = Yes, Thriller = No, Comedy = No, Crime = No,
Mystery = No, History = Yes

In [155]:
!pip install fuzzywuzzy
#import fuzzywuzzy

In [156]:
#use fuzzy matching to match two strings together regardless of the spelling or caps error
from fuzzywuzzy import process

In [157]:
#Make recommendation movie

def make_recommendation(movie_name, data, model, n_recommendation):
  #data = matrix or features
  #model = fit model
  #n_neighbors = how many recommendation do we need
  model.fit(data)

  #extractOne is to extract one row 
  # [] is to extract the index from print(id)
  #print(id)
  #the value of id = process.extractOne(movie_name, movie_idx['Movie Name']) = ('movie name', 'movie id', index)
  id = process.extractOne(movie_name, movie_idx['Movie Name'])[2]
  print('Selected Movie: ', movie_idx['Movie Name'][id])
  print('Searching...... \n')
  print('Recommended Movies: ')

  #kneighbors([X, n_neighbors, return_distance]): Finds the K-neighbors of a point.
  #data[id] = to take one movie and find the similar movies available in the dataset

  #model.kneighbors return two arrays: distance and movie index
  #distance is to show a distance from one point to another using the metrics
  distance, indexes = model.kneighbors(data[id], n_neighbors = n_recommendation)

  #within all the indexes, print all the movie names. 
  #.where(i!=id)) = where the index is not equal to the index that we used under 'id' so that the selected movie is not comparing with itself. 
  for i in indexes:
    movie = pd.DataFrame(movie_idx['Movie Name'][i].where(i!=id)).reset_index(drop=True)
  distances = pd.DataFrame(distance, index=['Distance']).transpose()
  movie_distance = pd.concat([movie, distances], axis=1)
  # changing to array movie_distance = list(movie_distance).to_numpy()
  return movie_distance

 
make_recommendation('the post', match_features, model_knn, 6)

Selected Movie:  The Post
Searching...... 

Recommended Movies: 


,Movie Name,Distance
0,NaN,0.000000
1,12 Years a Slave,0.900000
2,Hacksaw Ridge,1.000000
3,Queen of Katwe,1.019804
4,The Wind Rises,1.166190
5,A Beautiful Mind,1.414214
